In [1]:
import cv2
import perception
import matplotlib.pyplot as plt
import os
import json
import time
import numpy as np
import PoseComposer as pc
import random
from ctu_crs import CRS97
from numpy import array

DATA_FOLDER = "data"

In [2]:
robot = CRS97()
robot.initialize(home=False)

Firmware version : MARS8 v1.0 build Pi Oct  2 2017 11:06:45


In [15]:
robot.soft_home()

In [33]:
# points generator
number = 12
xmin, xmax = 0.35, 0.55
ymin, ymax = -0.22, 0.22
zmin, zmax = 0.1, 0.4

points = [
    np.array([
        random.uniform(xmin, xmax),
        random.uniform(ymin, ymax),
        random.uniform(zmin, zmax)
    ])
    for _ in range(number)
]

points

[array([0.54634127, 0.16892484, 0.32586135]),
 array([ 0.54812552, -0.05027511,  0.2838893 ]),
 array([ 0.48401687, -0.06885186,  0.27913709]),
 array([0.44678267, 0.1831153 , 0.16355492]),
 array([ 0.37842816, -0.20320066,  0.37296066]),
 array([0.50691086, 0.17527872, 0.20693413]),
 array([0.54468655, 0.00684527, 0.12936144]),
 array([ 0.51553321, -0.08592977,  0.25298888]),
 array([ 0.51565632, -0.16149795,  0.31313925]),
 array([ 0.54909339, -0.09099   ,  0.28698401]),
 array([ 0.41528241, -0.03221173,  0.30910623]),
 array([ 0.49170296, -0.20067758,  0.3184832 ])]

In [34]:
moves = []
for i in range(number):
    normal_vector = np.array([0, 0, 1])
    point = points[i]
    direction = np.array([0.45 - point[0], 0 - point[1] , 0])
    moves.append(pc.make_se3_matrix(point, direction, normal_vector))
    

In [35]:
def move_robot_to(position):
    q0 = robot.q_home
    ik_sols = robot.ik(position)
    assert len(ik_sols) > 0
    closest_solution = min(ik_sols, key=lambda q: np.linalg.norm(q - q0))
    robot.move_to_q(closest_solution)
    robot.wait_for_motion_stop()

In [36]:
for i in range(number):
    move_robot_to(moves[i])
    while robot.in_motion():
        continue
    img = robot.grab_image()
    cv2.imwrite(f"data/image{i}.png", img)
robot.soft_home()

In [32]:
robot.soft_home()

In [37]:


def find_circle(img):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Blur to reduce noise
    gray = cv2.GaussianBlur(gray, (9, 9), 1.5)

    # Detect circles
    circles = cv2.HoughCircles(
        gray,
        cv2.HOUGH_GRADIENT,
        dp=1.2,
        minDist=50,
        param1=100,
        param2=30,
        minRadius=5,
        maxRadius=0
    )

    if circles is None:
        raise ValueError("No circle detected")

    # Take the strongest detected circle
    circles = np.uint16(np.around(circles))
    x, y, r = circles[0][0]

    return np.array([x, y, r])

In [13]:
# "photos"
# files = os.listdir("photos")
# files

['foto5.jpg',
 'foto1.jpg',
 'foto6.jpg',
 'foto3.jpg',
 'foto4.jpg',
 'foto2.jpg',
 'foto7.jpg']

In [46]:
# DATA_FOLDER = "photos"
files = os.listdir(DATA_FOLDER)
# print(files)

robot_xyz = points
image_uv = []
imagenums = {}
errors = []
for f in files:
    file_name = f.split(".")[0]
    img = cv2.imread(os.path.join(DATA_FOLDER, f))
    try:
        c = find_circle(img)
    except:
        print(f"ERROR: image{f} cant detect circle")
        errors.append(f)
        continue
    xy = c[:2]
    radius = c[2]
    cv2.circle(img, center=np.array(xy, dtype = int), radius=5, thickness=20, color=(255, 0, 255) )
    cv2.imwrite(os.path.join("augumented_data", f"{file_name}_a.jpg"), img)
    # cv2.imwrite("a.jpg", img)
    image_uv.append(xy)

    imagenums[file_name] = xy
    print(file_name, xy)
    # time.sleep(1)
    # plt.imshow(img) 


# with open("image_uv.json", "w") as f:
#     f.write(json.dumps(image_uv))

# with open("robot_xyz.json", "w") as f:
#     f.write(json.dumps(robot_xyz))



image7 [596 927]
image8 [164 927]
image3 [1869  628]
image4 [1159  668]
image12 [1501 1063]
image10 [893 398]
image11 [746 297]
image9 [ 550 1106]
image2 [683 770]
image0 [1623 1121]
image5 [1861  914]
image6 [1045 1047]
image1 [ 758 1111]


In [47]:
errors

[]

In [48]:
imagenums

{'image7': array([596, 927], dtype=uint16),
 'image8': array([164, 927], dtype=uint16),
 'image3': array([1869,  628], dtype=uint16),
 'image4': array([1159,  668], dtype=uint16),
 'image12': array([1501, 1063], dtype=uint16),
 'image10': array([893, 398], dtype=uint16),
 'image11': array([746, 297], dtype=uint16),
 'image9': array([ 550, 1106], dtype=uint16),
 'image2': array([683, 770], dtype=uint16),
 'image0': array([1623, 1121], dtype=uint16),
 'image5': array([1861,  914], dtype=uint16),
 'image6': array([1045, 1047], dtype=uint16),
 'image1': array([ 758, 1111], dtype=uint16)}

In [49]:
# image_uv 18 is wrong

In [50]:
image_uv = []
robot_xyz = []
for i in range(number):
    if i != 18 and f"image{i}" in imagenums:
        image_uv.append(imagenums[f"image{i}"])
        robot_xyz.append(points[i])
    

In [4]:
image_uv = [[1623, 1121],
 [ 758, 1111],
 [683, 770],
 [1869,  628],
 [1159,  668],
 [1861,  914],
 [1045, 1047],
 [596, 927],
 [164, 927],
 [ 550, 1106],
 [893, 398],
 [746, 297]]

In [5]:
robot_xyz = [[0.54634127, 0.16892484, 0.32586135],
 [ 0.54812552, -0.05027511,  0.2838893 ],
 [ 0.48401687, -0.06885186,  0.27913709],
 [0.44678267, 0.1831153 , 0.16355492],
 [ 0.37842816, -0.20320066,  0.37296066],
 [0.50691086, 0.17527872, 0.20693413],
 [0.54468655, 0.00684527, 0.12936144],
 [ 0.51553321, -0.08592977,  0.25298888],
 [ 0.51565632, -0.16149795,  0.31313925],
 [ 0.54909339, -0.09099   ,  0.28698401],
 [ 0.41528241, -0.03221173,  0.30910623],
 [ 0.49170296, -0.20067758,  0.3184832 ]]

In [6]:
len(image_uv) == len(robot_xyz)

True

In [9]:




calibration_path = "camera_intrinsics.npz"
calibration_data = np.load("camera_intrinsics.npz")
camera_matrix = calibration_data["camera_matrix"]
dist_coeffs = calibration_data["dist_coeffs"]

obj = np.array(robot_xyz, dtype=np.float64).reshape(-1, 3)
obj_mm = obj * 1000.0
img = np.array(image_uv, dtype=np.float64).reshape(-1, 2)

K = camera_matrix
dist = dist_coeffs

ok, rvec, tvec, inliers = cv2.solvePnPRansac(
    obj_mm, img, K, dist,
    flags=cv2.SOLVEPNP_ITERATIVE,
    reprojectionError=3.0,
    iterationsCount=200
)
assert ok

obj_in = obj[inliers[:,0]]
img_in = img[inliers[:,0]]
ok, rvec, tvec = cv2.solvePnP(obj_in, img_in, K, dist, rvec, tvec, True)

R, _ = cv2.Rodrigues(rvec)

# kamera -> robot
R_RC = R.T
t_RC = -R.T @ tvec

T_RC = np.eye(4)
T_RC[:3,:3] = R_RC
T_RC[:3, 3] = t_RC.ravel()
print(T_RC)
T_RC
# [[-0.05876807,  0.99562368, -0.07266219,  0.50924364],
#        [ 0.99808416,  0.05719049, -0.02360613, -0.01328189],
#        [-0.01934724, -0.07391027, -0.99707721,  1.20215746],
#        [ 0.        ,  0.        ,  0.        ,  1.        ]]
# [[-0.03198898,  0.99886673, -0.03524156,  0.47134595],
#        [ 0.99918346,  0.03108867, -0.02580536, -0.01114153],
#        [-0.0246805 , -0.03603827, -0.9990456 ,  1.20035971],
#        [ 0.        ,  0.        ,  0.        ,  1.        ]]

[[ 7.88260774e-01 -3.32689418e-01  5.17651141e-01 -4.72095354e+04]
 [ 3.09923994e-01  9.41400428e-01  1.33087764e-01 -2.18055063e+04]
 [-5.31593896e-01  5.55246452e-02  8.45177463e-01 -8.13972659e+04]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


array([[ 7.88260774e-01, -3.32689418e-01,  5.17651141e-01,
        -4.72095354e+04],
       [ 3.09923994e-01,  9.41400428e-01,  1.33087764e-01,
        -2.18055063e+04],
       [-5.31593896e-01,  5.55246452e-02,  8.45177463e-01,
        -8.13972659e+04],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [11]:
import camera
cam = camera.Camera()

In [14]:
obj = np.asarray(robot_xyz, dtype=np.float64).reshape(-1, 3)
obj_mm = obj * 1000.0
img = np.asarray(image_uv, dtype=np.float64).reshape(-1, 2)
cam.transformFromCameraToRobot(obj_mm, img)

[[-3.09275095e-02  9.98889991e-01 -3.55285092e-02  4.71849312e+02]
 [ 9.99249525e-01  3.00701923e-02 -2.44165920e-02 -1.25522968e+01]
 [-2.33211402e-02 -3.62569903e-02 -9.99070345e-01  1.19959167e+03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
